In [127]:
import pandas
START_DATE = '2011-01-01'
END_DATE = '2017-01-01'

def preprocess(filename, start_date, end_date):
    TradingDaysPeriods = [1, 2, 3, 5, 10, 20, 40, 60, 120, 250]
    
    data = pandas.read_csv(filename, delimiter=',')
    data['Date'] = pd.to_datetime(data['Date'])

    return_features = pandas.DataFrame()
    volume_features = pandas.DataFrame()
    for period in TradingDaysPeriods:
        column = data['Adj Close'].pct_change(periods = period)
        column.rename("Return_" + str(period), inplace = True)
        return_features[column.name] = column
        
        column = data['Volume'].rolling(period).mean()
        column.rename("AverageVolume_" + str(period), inplace = True)
        volume_features[column.name] = column

    out_data = pandas.concat([return_features, volume_features], axis=1)
    out_data.insert(0, 'Date', data['Date'])
    out_data['DayOfWeek'] = out_data['Date'].dt.dayofweek
    out_data['DayOfMonth'] = out_data['Date'].dt.day
    
    #https://stackoverflow.com/questions/29370057/select-dataframe-rows-between-two-dates
    mask = (out_data['Date'] >= start_date) & (out_data['Date'] <= end_date)
    out_data = out_data.loc[mask]
    out_data.to_csv(path_or_buf = "features_" + filename, index=False, sep=';') #, sep=';'
    
preprocess('SPY.csv', START_DATE, END_DATE)
